In [1]:
import numpy as np
import pylab
import tensorflow as tf
import os

In [2]:
!nvidia-smi

Fri May 10 03:04:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:01:00.0  On |                  N/A |
| 23%   34C    P5    21W / 250W |    457MiB / 12186MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17118137069235598313
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11453762765
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7011968063162370535
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [6]:
cd /home/monuki/Documents/AI_code/CropPadTest5buff

/home/monuki/Documents/AI_code/CropPadTest5buff


In [7]:
!ls

0test  Book2.csv	     labels.npy  stacks		   UCI-43-18 raw.tif
1test  CropPadTest5Buff.npy  MIPs	 UCI-43-18_C3.tif


In [8]:
x = np.load('CropPadTest5Buff.npy')
y = np.load('labels.npy')
print(x.shape)
print(y.shape)

(802, 64, 64, 64, 3)
(802, 1)


In [9]:
def create_model():
    """
    Method to create the following CNN architecture:
    
      - BLOCK 1 (filter depth of 8) 64
      - BLOCK 2 (filter depth of 16) 32
      - BLOCK 3 (filter depth of 32) 16
      - BLOCK 4 (filter depth of 64) 8
      - RESHAPE
      - HIDDEN LAYER (128 nodes)
      - LOGIT SCORES (10 nodes)
      
    """
    # Reset our graph to build a new one
    tf.reset_default_graph()

    # ------------------------------------------------------------------------
    # Define placeholders for our images and labels
    # ------------------------------------------------------------------------
    #
    # 1. Define images
    # 
    #  - type: float32
    #  - size: [None, 32, 32, 3] so that we can feed in as many images as we need
    # 
    # 2. Define labels
    # 
    #  - type: int64
    #  - size: [None] so that we can feed in as many labels as we need
    # 
    # ------------------------------------------------------------------------

    im = tf.placeholder(tf.float32, [None, 64, 64, 64, 3])
    #how to deal with 3d images????
    labels = tf.placeholder(tf.int64, [None])
    #labels = tf.placeholder(tf.int64, )
    # ------------------------------------------------------------------------
    # Define convolutional blocks 
    # ------------------------------------------------------------------------
    #
    # As in previous assignments, we will use the tf.get_variables(...) method
    # to create matrix variables initialized to random values.
    # 
    # ------------------------------------------------------------------------

    # Block 1 (8 filter depth) 64^3 input
    w1 = tf.get_variable('w1', [3, 3, 3, 3, 2], dtype=tf.float32)
    w2 = tf.get_variable('w2', [3, 3, 3, 2, 2], dtype=tf.float32)
    layer = tf.nn.relu(tf.nn.conv3d(im, w1, strides=[1, 1, 1, 1, 1], padding='SAME'))
    layer = tf.nn.relu(tf.nn.conv3d(layer, w2, strides=[1, 2, 2, 2, 1], padding='SAME'))

    # Block 2 (16 filter depth) 32^3 input
    w3 = tf.get_variable('w3', [3, 3, 3, 2, 4], dtype=tf.float32)
    w4 = tf.get_variable('w4', [3, 3, 3, 4, 4], dtype=tf.float32)
    layer = tf.nn.relu(tf.nn.conv3d(layer, w3, strides=[1, 1, 1, 1, 1], padding='SAME'))
    layer = tf.nn.relu(tf.nn.conv3d(layer, w4, strides=[1, 2, 2, 2, 1], padding='SAME'))
    
    # Block 3 (32 filter depth) 16^3 input
    w5 = tf.get_variable('w5', [3, 3, 3, 4, 8], dtype=tf.float32)
    w6 = tf.get_variable('w6', [3, 3, 3, 8, 8], dtype=tf.float32)
    layer = tf.nn.relu(tf.nn.conv3d(layer, w5, strides=[1, 1, 1, 1, 1], padding='SAME'))
    layer = tf.nn.relu(tf.nn.conv3d(layer, w6, strides=[1, 2, 2, 2, 1], padding='SAME'))
    
    # Block 4 (64 filter depth) 8^3 input
    w7 = tf.get_variable('w7', [3, 3, 3, 8, 16], dtype=tf.float32)
    w8 = tf.get_variable('w8', [3, 3, 3, 16, 16], dtype=tf.float32)
    layer = tf.nn.relu(tf.nn.conv3d(layer, w7, strides=[1, 1, 1, 1, 1], padding='SAME'))
    layer = tf.nn.relu(tf.nn.conv3d(layer, w8, strides=[1, 2, 2, 2, 1], padding='SAME'))
    
    
    # ------------------------------------------------------------------------
    # Reshape to 1D vector 
    # ------------------------------------------------------------------------
    
    flattened = tf.reshape(layer, [-1, 4 * 4 * 4 * 16])

    # ------------------------------------------------------------------------
    # Define our matmul operations
    # ------------------------------------------------------------------------

    # Hidden layer (128 nodes)
    ### got rid of based on advise from peter
    
    #w9 = tf.get_variable('w9', [4 * 4 * 4 * 512, 128], dtype=tf.float32)
    #h1 = tf.nn.relu(tf.matmul(flattened, w9))
    
    # Logits (2 nodes)
    w9 = tf.get_variable('w9', [4 * 4 * 4 * 16, 2], dtype=tf.float32)
    logits = tf.matmul(flattened, w9)

    # ------------------------------------------------------------------------
    # Define our softmax cross-entropy loss
    # ------------------------------------------------------------------------
    # 
    # HINT: use tf.losses.sparse_softmax_cross_entropy() as above
    #
    # ------------------------------------------------------------------------

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # ------------------------------------------------------------------------
    # Define our optimizer
    # ------------------------------------------------------------------------

    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    return im, labels, [w1, w2, w3, w4, w5, w6, w7, w8, w9], logits, loss, train_op

# ------------------------------------------------------------------------
# Test our model
# ------------------------------------------------------------------------
# 
# If the graph was defined properly, we should be able to check the out
# what the model outputs should look like. Can you guess by the shapes
# of our logits and losses will be?
# 
# ------------------------------------------------------------------------

im, labels, weights, logits, loss, train_op = create_model()
print(logits.shape)
print(loss.shape)

(?, 2)
()


In [10]:
# ------------------------------------------------------------------------
# Create our model
# ------------------------------------------------------------------------

im, labels, weights, logits, loss, train_op = create_model()

# ------------------------------------------------------------------------
# Add to collections
# ------------------------------------------------------------------------
# 
# Collections are used by TensorFlow to keep track of certain intermediate 
# values for quick access during save/load functions.
# 
# ------------------------------------------------------------------------

tf.add_to_collection('im', im)
tf.add_to_collection('logits', logits)

# ------------------------------------------------------------------------
# Initialize our test graph
# ------------------------------------------------------------------------
# 
# What two things do we need to initialize our graph?
# 
# ------------------------------------------------------------------------

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# ------------------------------------------------------------------------
# Initialize our test graph
# ------------------------------------------------------------------------
# 
# Initialize a Saver object
# 
# ------------------------------------------------------------------------

saver = tf.train.Saver()

In [11]:
# ------------------------------------------------------------------------
# Train our algorithm 
# ------------------------------------------------------------------------
# 
# Let's set up a loop to train our algorithm by feeding it data iteratively.
# For each iteration, we will feed a batch_size number of images into our 
# model and let it readjust it's neuronal weights.
# 
# ------------------------------------------------------------------------

def train_model(iterations=10, batch_size=4):
    
    accuracies = []
    losses = []

    for i in range(iterations):

        # --------------------------------------------------------------------
        # Grab a total of batch_size number of random images and labels 
        # --------------------------------------------------------------------
        # 
        # 1. Pick batch_size number of random indices between 0 and 50,000
        # 2. Select those images / labels
        #
        # --------------------------------------------------------------------

        rand_indices = np.random.randint(802, size=(batch_size))
        x_batch = x[rand_indices]
        y_batch = y[rand_indices]
        y_batch = y_batch[..., 0]

        # --------------------------------------------------------------------
        # Normalize x_batch
        # --------------------------------------------------------------------
        # 
        # Currently, values in x range from 0 to 255. If we normalize these values
        # to a mean of 0 and SD of 1 we will improve the stability of training
        # and furthermore improve interpretation of learned weights. Use the
        # following code to normalize your batch:
        # 
        # --------------------------------------------------------------------

        x_batch = (x_batch - np.mean(x_batch)) / np.std(x_batch)

        # Convert to types matching our defined placeholders
        x_batch = x_batch.astype('float32')
        y_batch = y_batch.astype('int64')

        # Prepare feed_dict
        feed_dict = {im: x_batch, labels: y_batch}

        # --------------------------------------------------------------------
        # Run training iteration via sess.run()
        # --------------------------------------------------------------------
        # 
        # Here, in addition to whichever ouputs we wish to extract, we need to
        # also include the train_op variable. Including train_op will tell 
        # Tensorflow that in addition to calculating the intermediates of our graph,
        # we also need to readjust the variables so that the overall loss goes
        # down.
        # 
        # --------------------------------------------------------------------

        outputs = sess.run([logits, loss, train_op], feed_dict)

        # --------------------------------------------------------------------
        # Use argmax to determine highest logit (model guess)
        # --------------------------------------------------------------------
        # 
        # Keep in mind our logits matrix is (batch_size x 10) in size representing
        # a total of batch_size number of predictions. How do we process this matrix
        # with the np.argmax() to find the highest logit along each row of the matrix
        # (e.g. find the prediction for each of our images)?
        # 
        # HINT: what does the axis parameter in np.argmax(a, axis) specify?
        # 
        # --------------------------------------------------------------------

        predictions = np.argmax(outputs[0], axis=1)

        # --------------------------------------------------------------------
        # Calculate accuracy 
        # --------------------------------------------------------------------
        # 
        # Consider the following:
        # 
        # - predictions = the predicted digits
        # - y_batch = the ground-truth digits
        # 
        # How do I calculate an accuracy % with this data?
        # 
        # --------------------------------------------------------------------

        accuracy = np.sum(predictions == y_batch) / batch_size

        # --------------------------------------------------------------------
        # Accumulate and print iteration, loss and accuracy 
        # --------------------------------------------------------------------

        print('Iteration %05i | Loss = %07.3f | Accuracy = %0.4f' %
            (i + 1, outputs[1], accuracy))

        losses.append(outputs[1])
        accuracies.append(accuracy)
        
    return losses, accuracies

In [ ]:
# --------------------------------------------------------------------
# Train model
# --------------------------------------------------------------------
losses, accuracies = train_model(iterations=10, batch_size=4)

# --------------------------------------------------------------------
# Graph outputs and accuracy
# --------------------------------------------------------------------

import pylab
pylab.plot(losses)
pylab.title('Model loss over time')
pylab.show()

pylab.plot(accuracies)
pylab.title('Model accuracy over time')
pylab.show()

# --------------------------------------------------------------------
# Save model
# --------------------------------------------------------------------
# 
# In this step, all model variables and the underlying graph structure
# are saved so that they can be reloaded. Although it looks like just one
# file is saved here, in fact both a *.cpkt and *.cpkt.meta file are both
# saved in this single line of code.
#  
# --------------------------------------------------------------------

import os
model_file = './model_cnn3d_test/model.ckpt'
os.makedirs(os.path.dirname(model_file), exist_ok=True)
print('Saving model')
saver.save(sess, model_file)